In [1]:
# Rule : Get the background of user first

In [2]:
# Obtain the background of the indivudual user

In [3]:
# Rule-based Chatbot

# 1. [bot_entity] ชื่ออะไร -> if bot_entity in ['นาย','คุณ','เธอ'] then ผมชื่อ guruplan
# 2. อยาก[รู้|ทราบ]รายละเอียดคอร์ส [plan_name] -> return [plan_name] มีรายละเอียดดังกล่าว info_dict[plan_name] -> current_plan 
# 3. คอร์สนี้ราคาเท่าไร -> get(user.plan) -> p = info_dict[plan_name] -> ราคา {p}

In [4]:
class UserState:
    def __init__(self,name,age):
        self.name = name
        self.age = age
        self.course_name = None
        
    def update_course(self,course_name):
        self.course_name = course_name

In [8]:
import re, json

pairs = {r'.*สวัสดี.*': 'สวัสดีครับ {user_name}',
        r'(?:คุณ|เธอ|นาย)ชื่ออะไร': 'ผมชื่อ {bot_name} งับ',
         r'.*เข้าใจไรบ้าง.*': 'คุณสามารถถามฉันเรื่องคอร์สได้',
         r'.*(?:ขอ|อยากรู้|อยากทราบ|อยากได้)?(?:รายละเอียด|ข้อมูล)คอร์ส(.*)': 'รายละเอียดเกี่ยวกับคอร์ส {course} มีดังนี้ครับ : {course_info}',
         r'.*คอร์สนี้ราคาเท่าไร.*':'{price}',
        r'.*(?:ผม|หนู|กู)อายุเท่าไร(.*)': 'คุณ {user_name} อายุ {user_age} ปี',
        r'.*มีคอร์ส(?:อะไร|ไร).*' : 'มีคอร์สตามนี่ครับ \n {all_course_name}'} 

with open('rule.json',encoding="utf8") as f:
    rule = json.load(f)

user_state = UserState('Run',22)

def chat(query,pairs = pairs,rule = rule,user_state = user_state):

    for pattern, response in pairs.items():
        match = re.match(pattern, query)
        if match:
            if '{bot_name}' in response:
                name = 'rulebot'
                response = re.sub(r'\{bot_name\}', name, response)
                
            if '{course}' in response:
                course = match.group(1)
                course = course.strip()
                response = re.sub(r'\{course\}', course, response)
            
            if '{course_info}' in response:
                course_info = rule['course_info'].get(course,None)
                if course_info:
                    response = re.sub(r'\{course_info\}', course_info, response)
                    user_state.update_course(course)
                else:
                    response = f'ไม่มีคอร์สชื่อ "{course}" นะครับ 😞'
                
            if '{user_name}'in response:
                response = re.sub(r'\{user_name\}', user_state.name  , response)
            
            if '{user_age}'in response:
                response = re.sub(r'\{user_age\}', str(user_state.age)  , response)
            
            if '{price}'in response:
                if user_state.course_name:
                    course_info = rule['course_info'].get(user_state.course_name,None)
                    pattern = re.compile('ราคา \d+ บาท')
                    price = pattern.search(course_info)
                    response = re.sub(r'\{price\}', price.group(), response)
                    
                else:
                    response = f'คุณ {user_state.name} หมายถึงคอร์สไหนเหรอ'
                    
            if '{all_course_name}'in response:
                response = re.sub(r'\{all_course_name\}', ''  , response)   
                for course_ in rule['course_info']:
                    response += f'\n> {course_}'
                
            return response
            
    return 'ขอโทษที ฉันไม่เข้าใจ'

In [21]:
def interactive_chat(pairs = pairs,rule = rule,user_state = user_state):
    print('## หากต้องการจบการสนทนา พิมพ์ "พอ" หรือ "พักก่อน" ## \n')
    print(f'Rulebot 🤖 : สวัสดีคุณ {user_state.name} ผม rulebot มีอะไรให้ช่วยไหมครับเกี่ยวกับเรื่องคอร์สเรียนไหมครับ \n')
    while True:
        query = input(' You 👨 : ')
        if query in ['พอ','พักก่อน'] : 
            print(' Rulebot 🤖 : ขอบคุณมากที่ใช้บริการนะครับ แล้วเจอกันใหม่ครับ')
            break
        response = chat(query,pairs = pairs,rule = rule,user_state = user_state)
        print(f' Rulebot 🤖 : {response}')

In [9]:
interactive_chat()

## หากต้องการจบการสนทนา พิมพ์ "พอ" หรือ "พักก่อน" ## 

Rulebot 🤖 : สวัสดีคุณ Run ผม rulebot มีอะไรให้ช่วยไหมครับเกี่ยวกับเรื่องคอร์สเรียนไหมครับ 



 You 👨 :  สวะสดีครับ


 Rulebot 🤖 : ขอโทษที ฉันไม่เข้าใจ


 You 👨 :  สวัสดีครับ


 Rulebot 🤖 : สวัสดีครับ Run


...

 Rulebot 🤖 : ขอโทษที ฉันไม่เข้าใจ


 You 👨 :  555 โอเคงั้นเท่านี้ก่อนนะครับ


 Rulebot 🤖 : ขอโทษที ฉันไม่เข้าใจ


 You 👨 :  พักก่อน


 Rulebot 🤖 : ขอบคุณมากที่ใช้บริการนะครับ แล้วเจอกันใหม่ครับ
